In [1]:
%load_ext autoreload
%autoreload 2
import time
import tensorflow as tf
import p8_util

Using TensorFlow backend.


In [2]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
datadir = './data'
is_tensorboard = True

In [11]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR_TB)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [4]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (31083)kill: (31089)

In [5]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump


In [6]:
x_train.shape

(414, 224, 224, 3)

In [7]:
import p8_util_config
import NNAdaNetBuilder

#-----------------------------------------------------------------
# Get the number of convolutional layers for CNN network
# This will fixe the equivalent parameter of AdaNet num layers.
#-----------------------------------------------------------------
num_layers = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['conv_layer_num']
print("\n Num layers= {}".format(num_layers))
oNNAdaNetBuilder = NNAdaNetBuilder.NNAdaNetBuilder(p8_util_config.dict_adanet_config, num_layers=num_layers)


 Num layers= 2

*** NNAdaNetBuilder : NN Type=CNNBase


In [8]:
oNNAdaNetBuilder.show()



NN type              : ............................ CNNBase
Units in dense layer : ............................ 10
Number of layers     : ............................ 2
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Weights initializer  : ............................ xavier
Batch normalization  : ............................ True


CNN seed             : ............................ 42
Conv. Kernel size    : ............................ (3, 3)
Conv. layers         : ............................ 2
Dense layers         : ............................ 1
Units in conv. layers: ............................ 10
CNN bacth norm.      : ............................ True
Features map size    : ............................ [64]
Conv layers          : ............................ 2
Conv filters         : ............................ 32
Strides              : .......................

In [9]:
# Parameters that will be provided to model_fn
import p8_util_config


params = {'net_builder':oNNAdaNetBuilder}
nn_type = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type']
print("\n NN Type= {}".format(nn_type))
my_config = p8_util.make_config(nn_type,output_dir=OUTPUT_DIR, is_restored=False)  

classifier = tf.estimator.Estimator(model_fn=p8_util.my_model_fn, params=params, config=my_config)
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train, p8_util_config.NUM_EPOCHS\
                          , tuple_dimension=tuple_dimension\
                          , batch_size=p8_util_config.BATCH_SIZE),
        max_steps=p8_util_config.MAX_STEPS),
    
    eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("test", x_test, y_test, p8_util_config.NUM_EPOCHS\
                           , tuple_dimension=tuple_dimension\
                           , batch_size=p8_util_config.BATCH_SIZE),
        steps=None,
        throttle_secs=1))
end_time = time.time()


 NN Type= CNNBase
INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/CNNBase', '_tf_random_seed': 42, '_save_summary_steps': 5, '_save_checkpoints_steps': 5, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f16cba16048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and eva

INFO:tensorflow:Saving dict for global step 50: eval_accuracy = 0.23404256, global_step = 50, loss = 1.1136551
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 50: ./tmp/baseline/CNNBase/model.ckpt-50
INFO:tensorflow:Saving checkpoints for 55 into ./tmp/baseline/CNNBase/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-24-15:44:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/CNNBase/model.ckpt-55
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-24-15:44:12
INFO:tensorflow:Saving dict for global step 55: eval_accuracy = 0.25531915, global_step = 55, loss = 1.1184886
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 55: ./tmp/baseline/CNNBase/model.ckpt-55
INFO:tensorflow:Saving checkpoints for 60 into ./tmp/baseline/CNNBase/model.ckpt.
INFO:tensorf

In [10]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))



------------------------------------------------
Time (sec) 430.8435046672821
EVAL_ACCURACY: 0.3404255211353302
LOSS: 1.1684908866882324
GLOBAL_STEP: 100
